Импорты:

In [1]:
import numpy as np
import pandas as pd
import asyncio
import struct
import nest_asyncio
import bleak
from bleak import BleakScanner, BleakClient
from bleak.exc import BleakError
nest_asyncio.apply()
import re
from tqdm import tqdm

In [2]:
class BleakClientAssistant:
    def __init__(self, address, timeout=60):
        self.address = address
        self.timeout = timeout
        self.loop = asyncio.get_event_loop()
        self.client = None
        self.hk = None
        self.lk = None

    async def connect(self):
        self.client = BleakClient(self.address, timeout=self.timeout, cache_timeout=0)
        try:
            with tqdm(total=100, desc="Connecting and sending data") as pbar:
                success = await self.client.connect()
                for i in range(100):
                    pbar.update(1)
                    await asyncio.sleep(0.1)
                if success:
                    await asyncio.sleep(1)  
                    await self.client.start_notify(14, self.callback)
                else:
                    print("Failed to connect to device")
        except BleakError as e:
            print(f"Could not connect to device: {e}")
            retry = input("Would you like to retry connect? (y/n) ")
            if retry.lower() == "y":
                await self.connect()
            else:
                return
        
    async def disconnect(self):
        await self.client.stop_notify(14)
        await self.client.disconnect()
        
    async def send_message(self, message):
        await self.client.write_gatt_char(12, message)
        print(f'WRITE: {message} on TX_char')
        await asyncio.sleep(1)
        
    async def callback(self, sender: int, data: bytearray):
        response = data.decode().strip()
        print(f'Received notification from RX_char: {response}')
        match = re.search(r"HK:1:(\d+),LK:1:(\d+)", response)
        if match:
            self.hk = int(match.group(1))
            self.lk = int(match.group(2))
        
    def run(self, message):
        self.loop.run_until_complete(self.connect())
        self.loop.run_until_complete(self.send_message(message))
        self.loop.run_until_complete(self.disconnect())
        return self.hk, self.lk

In [3]:
def adv_decrypt(data):
    # Расшифровка уровня топлива
    oil_level_raw = int.from_bytes(data[1:3], byteorder='little', signed=False)
    print(f"Уровень топлива: {oil_level_raw}")

    # Расшифровка напряжения встроенной батареи
    battery_voltage_raw = int.from_bytes(data[3:4], byteorder='little', signed=False)
    battery_voltage = battery_voltage_raw / 10.0
    print(f"Напряжение встроенной батареи: {battery_voltage} В")

    # Расшифровка температуры
    TD_temp_raw = int.from_bytes(data[4:5], byteorder='little', signed=True)
    print(f"Температура: {TD_temp_raw} Градус Цельсия")

    return oil_level_raw, battery_voltage, TD_temp_raw

In [ ]:
class DataCollector:
    def __init__(self, start_serial, end_serial):
        self.start_serial = start_serial
        self.end_serial = end_serial
        self.device_names = ["TD_" + str(i) for i in range(start_serial, end_serial+1)]
        self.df = pd.DataFrame({'Name': self.device_names, 
                                'MAC': [None] * len(self.device_names),
                                'RSSI': [None] * len(self.device_names),
                                'O-L' : [None] * len(self.device_names),
                                'T': [None] * len(self.device_names),
                                'B-W': [None] * len(self.device_names),
                                'HK': [None] * len(self.device_names),
                                'LK': [None] * len(self.device_names)})
    def update_data()

In [4]:
class MyScanner:
    def __init__(self, timeout, start_serial, end_serial):
        self._scanner = BleakScanner(detection_callback=self.detection_callback)
        self.scanning = asyncio.Event()
        self.address = None
        self.timeout = timeout
        self.start_serial = start_serial
        self.end_serial = end_serial
        self.device_names = ["TD_" + str(i) for i in range(start_serial, end_serial+1)]
        self.df = pd.DataFrame({'Name': self.device_names, 
                                'MAC': [None] * len(self.device_names),
                                'RSSI': [None] * len(self.device_names),
                                'O-L' : [None] * len(self.device_names),
                                'T': [None] * len(self.device_names),
                                'B-W': [None] * len(self.device_names),
                                'HK': [None] * len(self.device_names),
                                'LK': [None] * len(self.device_names)})

        

    def detection_callback(self, device, advertisement_data):
        if device.name in self.device_names: 
            print(f'found device with {device.name} name')
            index = self.df.index[self.df['Name'] == device.name][0]
            if self.df.at[index, 'MAC'] is None:   
                self.df.at[index, 'MAC'] = device.address
            if self. df.at[index, 'RSSI'] is None:   
                self.df.at[index, 'RSSI'] = advertisement_data.rssi
            oil_level_raw, battery_voltage, TD_temp_raw = adv_decrypt(advertisement_data.manufacturer_data[3862])
            if self. df.at[index, 'O-L'] is None:   
                self.df.at[index, 'O-L'] = oil_level_raw
            if self. df.at[index, 'B-W'] is None:   
                self.df.at[index, 'B-W'] = battery_voltage   
            if self. df.at[index, 'T'] is None:   
                self.df.at[index, 'T'] = TD_temp_raw   
                
            assistant = BleakClientAssistant(device.address, timeout=self.timeout)
            hk, lk = assistant.run(b"GA\r")
            if self. df.at[index, 'HK'] is None:   
                self.df.at[index, 'HK'] = hk
            if self. df.at[index, 'LK'] is None:   
                self.df.at[index, 'LK'] = lk

    async def run(self):
        await self._scanner.start()
        self.scanning.set()
        end_time = loop.time() + self.timeout
        while self.scanning.is_set():
            if loop.time() > end_time:
                self.scanning.clear()
                print('\t\tScan has timed out so we terminate')
            await asyncio.sleep(0.1)
        await self._scanner.stop()

    def get_address(self):
        return self.address
    def get_dataframe(self):
        return self.df

In [7]:
my_scanner = MyScanner(timeout=30, start_serial=383448, end_serial=383449)
loop = asyncio.get_event_loop()
loop.run_until_complete(my_scanner.run())
address = my_scanner.get_address()

found device with TD_383449 name
Уровень топлива: 1
Напряжение встроенной батареи: 3.7 В
Температура: 22 Градус Цельсия


Connecting and sending data:   0%|                                                             | 0/100 [00:00<?, ?it/s]

found device with TD_383449 name
Уровень топлива: 1
Напряжение встроенной батареи: 3.7 В
Температура: 22 Градус Цельсия



Connecting and sending data:   0%|                                                             | 0/100 [00:00<?, ?it/s]

found device with TD_383448 name
Уровень топлива: 4095
Напряжение встроенной батареи: 3.7 В
Температура: 23 Градус Цельсия


...

Could not connect to device: Could not get GATT characteristics for <_bleak_winrt_Windows_Devices_Bluetooth_GenericAttributeProfile.GattDeviceService object at 0x0000023E9544FBB0>: Access Denied
Would you like to retry connect? (y/n) n


Exception in callback BleakScannerWinRT._received_handler(<_bleak_winrt...0023E958784B0>, <_bleak_winrt...0023E9587AAB0>)
handle: <Handle BleakScannerWinRT._received_handler(<_bleak_winrt...0023E958784B0>, <_bleak_winrt...0023E9587AAB0>)>
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Dukalis\AppData\Roaming\Python\Python310\site-packages\bleak\backends\winrt\scanner.py", line 208, in _received_handler
    self._callback(device, advertisement_data)
  File "C:\Users\Dukalis\AppData\Local\Temp\ipykernel_10020\3520866471.py", line 38, in detection_callback
    hk, lk = assistant.run(b"GA\r")
  File "C:\Users\Dukalis\AppData\Local\Temp\ipykernel_10020\2033048030.py", line 50, in run
    self.loop.run_until_complete(self.send_message(message))
  File "C:\ProgramData\anaconda3\lib\site-packages\nest_asyncio.py", line 90, in run_until_complete
    return f.result()

Could not connect to device: Device with address CA:17:37:73:61:98 was not found.
Would you like to retry connect? (y/n) n


Exception in callback BleakScannerWinRT._received_handler(<_bleak_winrt...0023E9540E630>, <_bleak_winrt...0023E9540E6F0>)
handle: <Handle BleakScannerWinRT._received_handler(<_bleak_winrt...0023E9540E630>, <_bleak_winrt...0023E9540E6F0>)>
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Dukalis\AppData\Roaming\Python\Python310\site-packages\bleak\backends\winrt\scanner.py", line 208, in _received_handler
    self._callback(device, advertisement_data)
  File "C:\Users\Dukalis\AppData\Local\Temp\ipykernel_10020\3520866471.py", line 38, in detection_callback
    hk, lk = assistant.run(b"GA\r")
  File "C:\Users\Dukalis\AppData\Local\Temp\ipykernel_10020\2033048030.py", line 50, in run
    self.loop.run_until_complete(self.send_message(message))
  File "C:\ProgramData\anaconda3\lib\site-packages\nest_asyncio.py", line 90, in run_until_complete
    return f.result()

WRITE: b'GA\r' on TX_char
Received notification from RX_char: AGA,UT:1:22,UL:1:1,HK:1:51941,LK:1:30474,US:1:30511,SE:1:383449,VB:2:3.7,WM:1:32768,TS:1:2110369,EM:
Received notification from RX_char: 1:0,TT:1:0,TM:1:10,UD:0:D9D9C5DDEEFF,VD:1:1,VV:1:141


In [8]:
my_scanner.get_dataframe()

,Name,MAC,RSSI,O-L,T,B-W,HK,LK
0,TD_383448,CA:17:37:73:61:98,-64,4095,23,3.7,None,None
1,TD_383449,E2:F1:F3:8C:E1:45,-54,1,22,3.7,51941,30474


In [58]:
MyScanner(1,100001,100010).get_dataframe()

,Name,MAC,RSSI,O-L,T,B-W,HK,LK
0,TD_100001,None,None,None,None,None,None,None
1,TD_100002,None,None,None,None,None,None,None
2,TD_100003,None,None,None,None,None,None,None
3,TD_100004,None,None,None,None,None,None,None
4,TD_100005,None,None,None,None,None,None,None
5,TD_100006,None,None,None,None,None,None,None
6,TD_100007,None,None,None,None,None,None,None
7,TD_100008,None,None,None,None,None,None,None
8,TD_100009,None,None,None,None,None,None,None
9,TD_100010,None,None,None,None,None,None,None


In [40]:
%%time
address = 'CA:17:37:73:61:98'
assistant = BleakClientAssistant(address, timeout=30)
hk, lk = assistant.run(b"GA\r")

Connecting and sending data: 100%|███████████████████████████████████████████████████| 100/100 [00:29<00:00,  3.36it/s]


WRITE: b'GA\r' on TX_char
Received notification from RX_char: AGA,UT:1:23,UL:1:4095,HK:1:30000,LK:1:10000,US:1:30693,SE:1:383448,VB:2:3.7,WM:1:32768,TS:1:2109144,
Received notification from RX_char: EM:1:0,TT:1:0,TM:1:10,UD:0:D8D9C5DDEEFF,VD:1:1,VV:1:141
CPU times: total: 1.44 s
Wall time: 38.2 s


In [35]:
print(f"HK: {hk}, LK: {lk}")

HK: 30000, LK: 10000
